## Utility functions

First we set up some utility functions to do bitwise XOR and encryption functions

In [1]:
from Crypto.Cipher import AES

def xor(ba1, ba2):
    return bytes([_a ^ _b for _a, _b in zip(ba1, ba2)])

def encrypt(data, key):
    nonce = b'sixteen.........'
    return AES.new(key, AES.MODE_EAX, nonce=nonce).encrypt(data)

def decrypt(ct, key):
    nonce = b'sixteen.........'
    return AES.new(key, AES.MODE_EAX, nonce=nonce).decrypt(ct)

## Generator operations
`c_0` and `c_1` are the two subcircuit branches.

`s_0` and `s_1` are the two labels representing possible values of the wire that determines the active branch.

We will create the material for `c_{0,1}` by encrypting with the label for the *other* subcircuit, `s_{1,0}`.

Finally, the generator will stack both branches, and send `m_cond = m_0 XOR m_1`



In [2]:
# adding junk at the end so the encryptions are more syntactically dissimilar
c_0, c_1 = b'subcircuit-0..qwe', b'subcircuit-1..iop' 
s_0, s_1 = b'label-0.000....q', b'label-1.111....w'

# encrypt c_0 with key s_1
m_0 = encrypt(c_0, key=s_1)

# encrypt c_1 with key s_0
m_1 = encrypt(c_1, key=s_0)

# this is the material representing the stacked branches
m_cond = xor(m_0, m_1)

print(f'{m_0=}\n\n{m_1=}\n\n{m_cond=}')

m_0=b'7\x8b\x01\xd5v\x91\\\x147\x1b\xb1\x171\x02=,a'

m_1=b'D\x9f]\xdd\x11\xeb\xb4\xec\x92iE\xfa.q)S+'

m_cond=b's\x14\\\x08gz\xe8\xf8\xa5r\xf4\xed\x1fs\x14\x7fJ'


## Evaluator operations

The evaluator receives label `S` representing the branch to execute.  However, the evaluator does not know whether `S` equals `s_0` or `s_1`. Thus, the evaluator performs an execution under each assumption, retrieving two material results, one representing the active branch and the other being garbage. The evaluator cannot know which is which, so we need to do a oblivious demux. (TODO)

#### Here we assume branch 0 is the active branch

In [3]:
# assumme S = s_0
S = s_0

# encrypt c_1 with the presumptive s_0 key to recover m_0
eval_m_1 = encrypt(c_1, key=S)
candidate_m_0 = xor(m_cond, eval_m_1)
print('assuming S=s_0, we try to recover m_0:')
print(f'{candidate_m_0=}\nshould match\n{m_0=}')
print(f'>>> {candidate_m_0==m_0=}')

# if it turns out that S was not s_0, then we get garbage
S = s_1
eval_m_1_garbage = encrypt(c_1, key=S)
candidate_m_0_garbage = xor(m_cond, eval_m_1_garbage)
print('\nif we were wrong and S=s_1, we get garbage')
print(f'{candidate_m_0_garbage=}\nshould NOT match\n{m_0=}')
print(f'>>> {candidate_m_0_garbage==m_0=}')

assuming S=s_0, we try to recover m_0:
candidate_m_0=b'7\x8b\x01\xd5v\x91\\\x147\x1b\xb1\x171\x02=,a'
should match
m_0=b'7\x8b\x01\xd5v\x91\\\x147\x1b\xb1\x171\x02=,a'
>>> candidate_m_0==m_0=True

if we were wrong and S=s_1, we get garbage
candidate_m_0_garbage=b'D\x9f]\xdd\x11\xeb\xb4\xec\x92iE\xfb.q1K>'
should NOT match
m_0=b'7\x8b\x01\xd5v\x91\\\x147\x1b\xb1\x171\x02=,a'
>>> candidate_m_0_garbage==m_0=False


#### Here we assume branch 1 is the active branch

In [4]:
# assume S = s_1
S = s_1

# encrypt c_0 with the presumptive s_1 key to recover m_1
eval_m_0 = encrypt(c_0, key=S)
candidate_m_1 = xor(m_cond, eval_m_0)
print('assuming S=s_1, we try to recover m_1:')
print(f'{candidate_m_1=}\nshould match\n{m_1=}')
print(f'>>> {candidate_m_1==m_1=}')

# if it turns out that S was not s_1, then we get garbage
S = s_0
eval_m_0_garbage = encrypt(c_0, key=S)
candidate_m_1_garbage = xor(m_cond, eval_m_0_garbage)
print('\nif we were wrong and S=s_0, we get garbage')
print(f'{candidate_m_1_garbage=}\nshould NOT match\n{m_1=}')
print(f'>>> {candidate_m_1_garbage==m_1=}')

assuming S=s_1, we try to recover m_1:
candidate_m_1=b'D\x9f]\xdd\x11\xeb\xb4\xec\x92iE\xfa.q)S+'
should match
m_1=b'D\x9f]\xdd\x11\xeb\xb4\xec\x92iE\xfa.q)S+'
>>> candidate_m_1==m_1=True

if we were wrong and S=s_0, we get garbage
candidate_m_1_garbage=b'7\x8b\x01\xd5v\x91\\\x147\x1b\xb1\x161\x02%4t'
should NOT match
m_1=b'D\x9f]\xdd\x11\xeb\xb4\xec\x92iE\xfa.q)S+'
>>> candidate_m_1_garbage==m_1=False


## Sanity check
Confirm that candidates are what we expect (although the candidates match the original messages, so this is trivially true)

In [5]:
decrypt(candidate_m_1, key=s_0)

b'subcircuit-1..iop'

In [6]:
decrypt(candidate_m_0, key=s_1)

b'subcircuit-0..qwe'

## Demux
Recall that the Evaluator does not know which candidate is the active branch and which is garbage.  Here we do an oblivious demux to recover the active branch

In [7]:
# TODO